#  Convolutional NN it for facial keypoints recognition

Here I will build a convolutional neural network, and train it for the task of facial keypoints recognition. The data are obtained from Kaggle: **LINK**, and consists of **info**.

I will build the CNN using tensorflow **link**. 

##  Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, clear_output

from sklearn.model_selection import train_test_split
import tensorflow as tf

%matplotlib inline

##  Loading data

In [ ]:
training_data_full = pd.read_csv('training.csv')

In [ ]:
images = training_data_full['Image'].apply(lambda str_pic: np.array([int(px) for px in str_pic.split()]))

images = np.vstack([images.iloc[i] for i in range(len(images))])

In [ ]:
keypoints = training_data_full.drop('Image', axis = 1)

In [ ]:
keypoints.iloc[0]

In [ ]:
x_center = 44.420571
y_center = 57.066803
off = 3

plt.figure(figsize = (10,10))
plt.imshow(images[0].reshape(96,96), cmap = 'gist_gray')
plt.plot([x_center-off, x_center-off, x_center+off, x_center +off],
         [y_center-off,y_center + off,y_center-off,y_center+off ], 'b*')

## Including reflected pictures to the dataset

In order to increase our dataset I will reflect all images left to right, and add them as different images.

This implies flipping the images, and reflecting all x-coordinates of the keypoints. 

In [ ]:
# building an array of reflected images.
flipped_ims = np.zeros(images.shape)
for j in range(images.shape[0]):
    for i in range(96):
        flipped_ims[j,i*96:(i+1)*96] = np.flip(images[j,i*96:(i+1)*96])

In [ ]:
# adding the reflected images to our images array. 
# RUN THIS CELL ONLY ONCE. 
images = np.vstack((images, flipped_ims))

In [ ]:
images.shape

In [ ]:
# Now the keypoints. I need to switch left and right features, 
# and reflect the x coordinates by x --> 95-x (95 is the last coordinate of the picture. )

reflected_keypoints = pd.DataFrame(columns = keypoints.columns)

# We look for the correspoding columns by switching 'left' <--> 'right'.
for colname in reflected_keypoints.columns:
    if 'left' in colname:
        reference_col = colname.replace('left', 'right')
    elif 'right' in colname:
        reference_col = colname.replace('right','left')
    else:
        reference_col = colname
        
    # Assigning values and reflecting x coordinates
    # reflected_keypoints[colname] = keypoints[reference_col].apply(lambda x: 95-x if colname[-1]=='x' else x)
    # the one-line version is fine but I think separating is more readable.
    
    reflected_keypoints[colname] = keypoints[reference_col]
    if colname[-1] == 'x':
        reflected_keypoints[colname] = reflected_keypoints[colname].apply(lambda x: 95-x)   

In [ ]:
# Adding reflected keypoints to the original keypoints. 
# RUN THIS CELL ONLY ONCE.
keypoints = pd.concat([keypoints,reflected_keypoints], ignore_index=True)

## Perform a train_test_split in order to have a cross validation set. 

In [ ]:
print('Images', images.shape)
print('Keypoints', keypoints.shape)

In [ ]:
x_train, x_cv, keypoints_train, keypoints_cv = train_test_split(images, keypoints, 
                                                                test_size=100, random_state = 42)

##  Define a funaction for taking random batches. 

In [ ]:
def next_batch(X, labels, batch_size):
    
    sample_indices = np.random.choice(range(len(X)), size = batch_size, 
                                      replace = False)
    
    images = X[sample_indices]
    keypoints = labels.iloc[sample_indices]
        
    return images, keypoints

# Building the CNN

I will start by using the same architechture I used in the course.For the moment I will use train_test_split to test my network a bit. Eventually this wont be necessary, as the dataset provides a separate test set. 

In [ ]:
#Some helping functions.
def init_weights(shape):
    init_random_dist = tf.random_normal(shape, stddev=0.1) # Why this stddev?
    return tf.Variable(init_random_dist)

def init_bias(shape):
    init_bias_vals = tf.random_uniform(shape=shape)
    return tf.Variable(init_bias_vals)

def init_gamma(shape):
    init_gamma_val = tf.constant(1, dtype = tf.float32)
    return tf.Variable(init_gamma_val)

def init_beta(shape):
    init_beta_val = tf.constant(0, dtype = tf.float32)
    return tf.Variable(init_beta_val)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2by2(x):
    return tf.nn.max_pool(x, ksize=[1, 4, 4, 1],
                          strides=[1, 4, 4, 1], padding='SAME')

def convolutional_layer(input_x, shape):
    W = init_weights(shape)
    b = init_bias([shape[3]])
    return tf.nn.relu(conv2d(input_x, W) + b)

def normal_full_layer(input_layer, size):
    input_size = int(input_layer.get_shape()[1])
    W = init_weights([input_size, size])
    b = init_bias([size])
    return tf.matmul(input_layer, W) + b


def bn_layer(input_x, is_training, bn_type = 'normal', decay = 0.999):
    
    """Implements batch normalization. 
    input_x: A Tensor. Is the data to normalize.
    is_training: Bool. Indicates whether we are in training o predictino phase. 
    bn_type: 'normal' or 'convo', indicates over which indeces we take the means. 
              if 'normal', we use only the first index (batch number).
              if 'conv', uses [0,1,2]
    decay: float. decay parameter for the running averages. 
    """
    
    if bn_type == 'normal':
        shape = input_x.shape[1:]
        axes = [0]
    elif bn_type == 'convo':
        shape = input_x.shape[-1]
        axes = [0,1,2]
        
    #running mean and variance to be sued for inference.
    pop_mean = tf.Variable(tf.zeros(shape), trainable=False) 
    pop_var = tf.Variable(tf.ones(shape), trainable=False)
    
    gamma = init_gamma(shape) #in the normalization I am using this is the number of channels. 
    beta = init_beta(shape)
    
    batch_mean, batch_var = tf.nn.moments(input_x, axes)  
    
    def train_phase():
        train_mean = tf.assign(pop_mean, pop_mean * decay + batch_mean * (1 - decay))
        train_var = tf.assign(pop_var, pop_var * decay + batch_var * (1 - decay))
        with tf.control_dependencies([train_mean, train_var]):
            return tf.nn.batch_normalization(input_x, batch_mean, batch_var,
                                                    offset = beta, scale = gamma,
                                                    variance_epsilon=0.0001)
    
    def infer_phase(): 
        return tf.nn.batch_normalization(input_x, pop_mean, pop_var, offset = beta, 
                                  scale = gamma, variance_epsilon=0.0001)
    
        
    return tf.cond(is_training, train_phase, infer_phase)
    

In [ ]:
# PLACEHOLDERS

x = tf.placeholder(tf.float32, [None, 9216])
keypoints_true = tf.placeholder(tf.float32, [None, 30])
lr = tf.placeholder(tf.float32)
training = tf.placeholder(tf.bool)
drop_rate = tf.placeholder(tf.float32)

In [ ]:
x_images = tf.image.per_image_standardization(tf.reshape(x, [-1,96,96,1]))

In [ ]:
# LAYERS

convo1 = convolutional_layer(x_images, [8,8,1,32]) # 8 x 8 filter, 1 channel in, 32 channels out. SAME padding.
                                            # so output images are also 96 x 96. 

#convo1_bn = bn_layer(convo1, training, bn_type = 'normal')    

convo1_pool = max_pool_2by2(convo1)   #output of 24 x 24 x 32

# Image standarization. I guess it's the same before or after pooling. (maybe not!)
convo1_pin = tf.image.per_image_standardization(convo1_pool) 

convo2 = convolutional_layer(convo1_pin, [4,4,32,64]) # 4x4 filter, 64 outputs. SAME padding.

#convo2_bn = bn_layer(convo2, training, bn_type = 'normal')    

convo2_pool = max_pool_2by2(convo2) # 6 x 6 x 64

convo2_pin = tf.image.per_image_standardization(convo2_pool)

convo2_flat = tf.reshape(convo2_pin,[-1,6*6*64])

#convo2_flat_bn = bn_layer(convo2_flat, training, bn_type='normal')

full_layer_one = tf.nn.relu(normal_full_layer(convo2_flat,1024))


# DROPOUT AND OUTPOUT LAYER

full_one_dropout = tf.nn.dropout(full_layer_one, rate = drop_rate)

keypoints_pred = normal_full_layer(full_one_dropout,30)
#keypoints_pred = normal_full_layer(full_layer_one,30)

In [ ]:
# LOSS FUNCTION

masked_loss = tf.reduce_mean(tf.square(
        tf.boolean_mask(keypoints_pred-keypoints_true, tf.is_finite(keypoints_true) )
    ))


optimizer = tf.train.AdamOptimizer(learning_rate=lr)
train = optimizer.minimize(masked_loss)

In [ ]:
# INITIALIZER

init = tf.global_variables_initializer()

saver = tf.train.Saver()

In [ ]:
counter = 5

In [ ]:
keypoints_mean = keypoints_train.mean()

# Computing error from the mean in order to have a reference. This is the mark I have to beat. 

mean_keypoints_mse = ((keypoints_cv-keypoints_mean)**2).sum(axis = 1).mean()
mean_keypoints_mse

## Training

In [ ]:
num_steps = 5000

fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)

with tf.Session() as sess:
    
    #sess.run(init)
    saver.restore(sess, "./saved_models/keypoints_cnn_" + str(counter))
    train_losses = []
    cv_losses = []
    for iteration in range(num_steps+1):
        
        x_batch, keypoints_batch = next_batch(x_train, keypoints_train, 70)
        
        _ , train_loss = sess.run([train, masked_loss], 
                                  feed_dict={x: x_batch, keypoints_true: keypoints_batch,
                                             lr:0.0001, drop_rate:0.5})
        
        # PRINT OUT A MESSAGE EVERY 100 STEPS
        
        #if (iteration%20 == 0) and (iteration >0):
            #print(':')
        
        if iteration%100 == 0:
            
            cv_loss = sess.run(masked_loss,feed_dict={x:x_cv,keypoints_true:keypoints_cv,
                                                      drop_rate:0})
            
            train_losses.append(train_loss)
            cv_losses.append(cv_loss)
            
            # Showing output for tracking progress.
            print('=======================================')
            print('Currently on step {}'.format(iteration))
            print('Train MSE: ', train_loss, '\n')
            print('CV MSE:', cv_loss, '\n')
            
            ax.cla()
            ax.plot(train_losses[1:], 'r', label = 'Training error')
            ax.plot(cv_losses[1:], 'g', label = 'Validation error')
            ax.legend()
            display(fig)
            
            print('=======================================')
            print('train_losses = ', train_losses, '\n')
            print('cv_losses = ', cv_losses)
            clear_output(wait = True)
    
    # printing final results
    
    print('=======================================')
    print('Currently on step {}'.format(iteration))
    print('Train MSE: ', train_loss, '\n')
    print('CV MSE:', cv_loss, '\n')

    ax.cla()
    ax.plot(train_losses[1:], 'r', label = 'Training error')
    ax.plot(cv_losses[1:], 'g', label = 'Validation error')
    ax.legend()
    #display(fig)
            
    print('=======================================')
    print('train_losses = ', train_losses, '\n')
    print('cv_losses = ', cv_losses)
            
    saver.save(sess, "./saved_models/keypoints_cnn_" + str(counter) )
    #saver.save(sess, "./saved_models/keypoints_cnn_1"  )
    #counter +=1
    #print(counter)

## Predicting

In [ ]:
with tf.Session() as sess:
    
    # Use your Saver instance to restore your saved rnn time series model
    saver.restore(sess, "./saved_models/keypoints_cnn_1")

    # Create a numpy array for your genreative seed from the last 12 months of the 
    # training set data. Hint: Just use tail(12) and then pass it to an np.array
        
    predictions = sess.run(keypoints_pred, feed_dict= {x:x_cv, drop_prob:0 })
    

In [ ]:
predictions.shape

In [ ]:
x_cv.shape

In [ ]:
grid_size = 3
fig, axes = plt.subplots(grid_size, grid_size, gridspec_kw = dict(hspace = .05, wspace = .05), 
                         figsize=(10,10))

selection = np.random.choice(range(len(x_cv)), size = grid_size**2, )

mean_x_points = [keypoints_mean[j] for j in range(0,30,2)]
mean_y_points = [keypoints_mean[j+1] for j in range(0,30,2)]

for i, ax in zip(selection, axes.flat):

    ax.axis('off')
    # Plotting the faces
    ax.imshow(x_cv[i].reshape((96,96)),cmap='gist_gray')

 # Obtaining keypoints positions. x and y coordinates are even and odd indices respectively. 
    x_points = [predictions[i][j] for j in range(0,30,2)]
    y_points = [predictions[i][j+1] for j in range(0,30,2)]
      
    #plotting predicted keypoints
    ax.plot(x_points, y_points, 'ro', markerfacecolor = 'none')    
  
 # Plotting true keypoints
    
    x_true = [keypoints_cv.iloc[i][j] for j in range(0,30,2)]
    y_true = [keypoints_cv.iloc[i][j+1] for j in range(0,30,2)]
    
    ax.plot(x_true, y_true, 'y*', markerfacecolor = 'none')    
    
 # Including mean keypoints
       
    #ax.plot(mean_x_points, mean_y_points, 'b+', markerfacecolor = 'none')  

# Adding one more layer

In [ ]:
tf.reset_default_graph()

In [ ]:
# PLACEHOLDERS

x = tf.placeholder(tf.float32, [None, 9216])
keypoints_true = tf.placeholder(tf.float32, [None, 30])
lr = tf.placeholder(tf.float32)
training = tf.placeholder(tf.bool)
drop_rate = tf.placeholder(tf.float32)

In [ ]:
x_images = tf.image.per_image_standardization(tf.reshape(x, [-1,96,96,1]))

In [ ]:
# LAYERS

convo1 = convolutional_layer(x_images, [6,6,1,32]) # 6 x 6 filter, 1 channel in, 32 channels out. SAME padding.
                                            # so output images are also 96 x 96. 

convo1_pool = max_pool_2by2(convo1)   #output of 24 x 24 x 32

convo2 = convolutional_layer(convo1_pool, [2,2,32,64]) # 4x4 filter, 64 outputs. SAME padding.

convo2_pool = tf.nn.max_pool(convo2, ksize=[1, 2, 2, 1],
                          strides=[1, 2, 2, 1], padding='SAME') # 12 x 12 x64

convo3 = convolutional_layer(convo2_pool, [2,2,64,128]) # 2x2 filter, 128 outputs. SAME padding.

convo3_pool = tf.nn.max_pool(convo3, ksize=[1, 2, 2, 1],
                          strides=[1, 2, 2, 1], padding='SAME') # 6 x 6 x 128


convo3_flat = tf.reshape(convo3_pool,[-1,6*6*128])

full_layer_one = tf.nn.relu(normal_full_layer(convo3_flat,1024))

# DROPOUT AND OUTPOUT LAYER

full_one_dropout = tf.nn.dropout(full_layer_one, rate = drop_rate)

keypoints_pred = normal_full_layer(full_one_dropout,30)
#pre_pred_bn = bn_layer(full_layer_one, is_training =training)
#keypoints_pred = normal_full_layer(pre_pred_bn,30)


In [ ]:
# LOSS FUNCTION

masked_loss = tf.reduce_mean(tf.square(
        tf.boolean_mask(keypoints_pred-keypoints_true, tf.is_finite(keypoints_true) )
    ))


optimizer = tf.train.AdamOptimizer(learning_rate=lr)
train = optimizer.minimize(masked_loss)

In [ ]:
# INITIALIZER

init = tf.global_variables_initializer()

saver = tf.train.Saver()

In [ ]:
counter = 4

## Training

In [ ]:
num_steps = 10000

fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)

with tf.Session() as sess:
    
    #sess.run(init)
    saver.restore(sess, "./saved_models/keypoints_3_lay_cnn_" + str(counter))
    #saver.restore(sess, "./saved_models/keypoints_3_lay_cnn_3")
    train_losses = []
    cv_losses = []
    for iteration in range(num_steps+1):
        
        x_batch, keypoints_batch = next_batch(x_train, keypoints_train, 32)
        
        _ , train_loss = sess.run([train, masked_loss], 
                                  feed_dict={x: x_batch, keypoints_true: keypoints_batch,
                                             lr:0.0005, drop_rate:0})
        
        # PRINT OUT A MESSAGE EVERY 100 STEPS
        
        #if (iteration%20 == 0) and (iteration >0):
            #print(':')
        
        if iteration%100 == 0:
            
            cv_loss = sess.run(masked_loss,feed_dict={x:x_cv,keypoints_true:keypoints_cv,
                                                      drop_rate:0})
            
            train_losses.append(train_loss)
            cv_losses.append(cv_loss)
            
            # Showing output for tracking progress.
            print('=======================================')
            print('Currently on step {}'.format(iteration))
            print('Train MSE: ', train_loss, '\n')
            print('CV MSE:', cv_loss, '\n')
            
            ax.cla()
            ax.plot(train_losses[1:], 'r', label = 'Training error')
            ax.plot(cv_losses[1:], 'g', label = 'Validation error')
            ax.legend()
            display(fig)
            
            print('=======================================')
            print('train_losses = ', train_losses, '\n')
            print('cv_losses = ', cv_losses)
            clear_output(wait = True)
    
    # printing final results
    
    print('=======================================')
    print('Currently on step {}'.format(iteration))
    print('Train MSE: ', train_loss, '\n')
    print('CV MSE:', cv_loss, '\n')

    ax.cla()
    ax.plot(train_losses[1:], 'r', label = 'Training error')
    ax.plot(cv_losses[1:], 'g', label = 'Validation error')
    ax.legend()
    
            
    print('=======================================')
    print('train_losses = ', train_losses, '\n')
    print('cv_losses = ', cv_losses)
            
    saver.save(sess, "./saved_models/keypoints_3_lay_cnn_" + str(counter) )
    #saver.save(sess, "./saved_models/keypoints_3_lay_cnn_1"  )
    #counter +=1
    #print(counter)

In [ ]:
train_losses =  [4558.8394, 20.908371, 17.039003, 20.049202, 10.044168, 9.703507, 15.559782, 6.865949, 
                 25.59113, 6.2370186, 8.187018, 11.429759, 7.894011, 7.7230015, 12.717848, 4.9590435, 
                 15.206853, 7.0373893, 6.0925264, 4.8623476, 7.5098534, 10.640968, 4.5147247, 6.9635158, 
                 7.5547614, 5.9242897, 19.583391, 4.6431746, 4.6215606, 4.2582717, 5.1475606, 4.7572646, 
                 6.7613974, 4.784758, 13.9355, 4.0508394, 13.597205, 5.513013, 5.189295, 5.959362, 25.297157, 
                 7.8781767, 5.3293223, 3.9638731, 4.6675124, 5.7606955, 5.559716, 4.909071, 3.2822552, 17.615404, 
                 4.542181, 6.3140674, 3.0813, 4.519869, 4.2858176, 4.862059, 3.2193875, 3.520087, 5.3996572, 
                 3.7363892, 3.3154879, 4.38009, 23.661266, 3.3384507, 3.1718805, 10.246564, 16.43965, 4.030565, 
                 2.8879497, 3.507426, 3.9944217, 3.9190907, 3.773814, 3.3246613, 2.9970434, 3.3631327, 3.5873342, 
                 3.4945877, 3.3367965, 5.1137104, 2.617096, 2.5338717, 2.362038, 4.124611, 2.9634113, 3.1306415, 
                 3.683314, 3.0808218, 10.531972, 2.6799278, 4.2612567, 3.0190673, 3.1299937, 2.5151055, 2.6948583, 
                 3.3608482, 19.846085, 2.6240127, 3.56997, 2.6583395, 2.4523296, 2.387698, 2.0636752, 3.2371287, 
                 7.678181, 3.111113, 2.1019063, 3.164981, 2.267844, 2.3275228, 3.404172, 2.417392, 2.529003, 
                 2.191953, 2.185136, 2.0412982, 3.2052617, 2.3686912, 2.2367973, 2.324498, 2.328194, 1.6763031, 
                 1.6991774, 2.32733, 1.9292554, 2.4200792, 1.6099848, 1.7967035, 2.2805612, 2.1812487, 1.7299813, 
                 12.77798, 2.5494215, 3.1983726, 1.6301839, 1.6257082, 10.110136, 1.8926824, 1.7287594, 2.3088117, 
                 1.6462872, 1.68596, 1.8830576, 1.6255205, 1.4220376, 1.3481195, 1.7550656, 1.8198619, 1.5103369, 
                 1.2812177, 1.5036436, 1.122376, 1.3292304, 1.5029494, 1.4890337, 1.3698753, 1.9358006, 1.6390549, 
                 1.180626, 1.380856, 1.3521146, 1.439276, 2.0195062, 1.3904762, 1.4819094, 1.5237277, 1.2113658, 
                 1.1957227, 1.4088675, 1.2315847, 1.6696317, 1.9314363, 1.4497982, 2.14771, 1.4954938, 1.253251,
                 1.48175, 1.9106281, 1.0101423, 1.0933654, 1.0932888, 1.1102728, 1.3479441, 0.79990846, 1.129885, 
                 1.1468667, 1.429645, 1.1123695, 1.1286811, 1.1239444, 0.9994459, 0.90256274, 1.1997993, 1.2634647, 
                 0.8119381, 1.0016273, 0.8939902, 1.0503857, 0.9986231, 1.5989772, 0.8234278, 0.9149763] 

cv_losses =  [1156.6577, 20.162817, 14.938917, 12.002618, 11.926132, 10.46158, 11.118082, 9.63501, 10.1692295, 
              8.926162, 9.299528, 9.380932, 8.600862, 8.575799, 8.961025, 8.041911, 8.846734, 7.791157, 8.280328, 
              7.754253, 8.008371, 8.693667, 7.25445, 8.215816, 7.488269, 7.535618, 7.452112, 7.0879016, 7.1619897, 
              7.349695, 7.611605, 6.794039, 8.285829, 6.426057, 6.8531404, 6.9880276, 7.0138865, 7.14933, 6.688554, 
              7.459748, 6.497299, 6.200159, 6.6876435, 6.2218103, 8.093535, 6.402516, 6.7375484, 6.023359, 5.9644556, 
              6.723415, 6.418011, 7.1418, 5.865808, 6.7843246, 5.844525, 6.499982, 5.7592187, 5.8337946, 5.5905747, 
              5.791144, 5.951072, 5.571767, 6.2703366, 5.4720354, 6.232392, 6.152627, 5.9544163, 5.285056, 5.282959, 
              6.05443, 4.9566345, 5.321445, 5.0027676, 4.6873217, 5.142541, 5.473357, 5.305813, 5.395688, 5.8638434, 
              5.0607204, 4.7306004, 4.5625834, 4.722171, 4.612138, 4.486454, 4.6587477, 4.4578333, 4.9510264, 
              5.0761805, 4.2348185, 5.0889926, 4.51063, 4.497944, 4.210184, 4.4326034, 4.906568, 4.647253, 4.187331, 
              3.8176403, 4.6061454, 4.24058, 4.321801, 3.9935093, 5.436344, 4.6540227, 3.927831, 4.205877, 4.247602, 
              4.170788, 4.100132, 3.9255872, 4.2058396, 3.6825593, 3.8364367, 3.697328, 3.9241922, 3.851854, 4.232018, 
              3.9288201, 3.865963, 4.239387, 3.533905, 3.6656184, 3.6761363, 3.6612766, 3.519028, 3.9420047, 3.6353176, 
              3.9629645, 5.248551, 3.5821958, 3.8398056, 3.9275012, 3.6335733, 3.7361848, 3.5296323, 4.3844953, 
              3.7799268, 3.5858893, 5.188334, 3.3746622, 3.8121102, 3.8102686, 3.6213355, 3.6777284, 3.591941, 
              3.6786795, 3.5151646, 3.3805044, 3.4434803, 3.66486, 3.2476742, 3.3852851, 3.6648674, 3.5789466, 
              3.6029296, 3.5007179, 3.355305, 3.6053352, 3.453718, 3.7212331, 3.6291282, 3.4063566, 3.6848967, 
              3.5452392, 3.7305937, 3.4374626, 3.442229, 3.481856, 3.3858092, 3.953431, 3.6834474, 4.169195, 
              4.2124524, 3.5008695, 3.6738503, 3.7179592, 3.5211384, 3.354789, 3.5371523, 3.824025, 3.7020347, 
              3.6464314, 3.751026, 3.6769652, 3.8978004, 3.9020684, 3.4495802, 3.680164, 3.617353, 3.5885813, 
              3.6612535, 3.5833437, 4.0460625, 3.6080875, 3.4751272, 3.63411, 3.7912266, 3.6112332, 3.4511962, 
              3.7310445, 3.5361388]

In [ ]:
np.sqrt(cv_losses[-1])

In [ ]:
counter

In [ ]:
with tf.Session() as sess:
    
    # Use your Saver instance to restore your saved rnn time series model
    saver.restore(sess, "./saved_models/keypoints_3_lay_cnn_" + str(counter))

    # Create a numpy array for your genreative seed from the last 12 months of the 
    # training set data. Hint: Just use tail(12) and then pass it to an np.array
        
    predictions = sess.run(keypoints_pred, feed_dict= {x:x_cv, drop_rate:0 })
    

In [ ]:
predictions.shape

In [ ]:
x_cv.shape

In [ ]:
grid_size = 3
fig, axes = plt.subplots(grid_size, grid_size, gridspec_kw = dict(hspace = .05, wspace = .05), 
                         figsize=(10,10))

selection = np.random.choice(range(len(x_cv)), size = grid_size**2, )

mean_x_points = [keypoints_mean[j] for j in range(0,30,2)]
mean_y_points = [keypoints_mean[j+1] for j in range(0,30,2)]

for i, ax in zip(selection, axes.flat):

    ax.axis('off')
    # Plotting the faces
    ax.imshow(x_cv[i].reshape((96,96)),cmap='gist_gray')

 # Obtaining keypoints positions. x and y coordinates are even and odd indices respectively. 
    x_points = [predictions[i][j] for j in range(0,30,2)]
    y_points = [predictions[i][j+1] for j in range(0,30,2)]
      
    #plotting predicted keypoints
    ax.plot(x_points, y_points, 'ro', markerfacecolor = 'none')    
  
 # Plotting true keypoints
    
    x_true = [keypoints_cv.iloc[i][j] for j in range(0,30,2)]
    y_true = [keypoints_cv.iloc[i][j+1] for j in range(0,30,2)]
    
    ax.plot(x_true, y_true, 'b*', markerfacecolor = 'none')    
    
 # Including mean keypoints
       
    #ax.plot(mean_x_points, mean_y_points, 'b+', markerfacecolor = 'none')  

## Predictions for the test set

In [ ]:
test_data = pd.read_csv('test.csv')
test_data.shape

In [ ]:
test_data.head()

In [ ]:
test_images = test_data['Image'].apply(lambda str_pic: np.array([int(px) for px in str_pic.split()]))

test_images = np.vstack([test_images.iloc[i] for i in range(len(test_images))])

In [ ]:
saver = tf.train.Saver()

In [ ]:
with tf.Session() as sess:
    
    
    
    # Use your Saver instance to restore your saved rnn time series model
    saver.restore(sess, "./saved_models/keypoints_3_lay_cnn_" + str(counter))

    # Create a numpy array for your genreative seed from the last 12 months of the 
    # training set data. Hint: Just use tail(12) and then pass it to an np.array
        
    predictions = sess.run(keypoints_pred, feed_dict= {x:test_images, drop_rate:0 })
    

In [ ]:
predictions.shape

In [ ]:
grid_size = 3
fig, axes = plt.subplots(grid_size, grid_size, gridspec_kw = dict(hspace = .05, wspace = .05), 
                         figsize=(10,10))

selection = np.random.choice(range(len(test_images)), size = grid_size**2, )

for i, ax in zip(selection, axes.flat):

    ax.axis('off')
    # Plotting the faces
    ax.imshow(test_images[i].reshape((96,96)),cmap='gist_gray')

 # Obtaining keypoints positions. x and y coordinates are even and odd indices respectively. 
    x_points = [predictions[i][j] for j in range(0,30,2)]
    y_points = [predictions[i][j+1] for j in range(0,30,2)]
      
    #plotting predicted keypoints
    ax.plot(x_points, y_points, 'ro', markerfacecolor = 'none')    

## Make a dataframe with predictions

In [ ]:
predicted_keypoints = pd.DataFrame(data = predictions, columns = keypoints.columns)
predicted_keypoints.info()

In [ ]:
predicted_keypoints.head()

In [ ]:
predicted_keypoints.to_csv('full_test_predictions.csv')

## Loading the lookup table to make a submission file. 

In [ ]:
lookup_table = pd.read_csv('IdLookupTable.csv', header = 0, index_col = 'RowId')

In [ ]:
lookup_table.head()

In [ ]:
lookup_table.info()

In [ ]:
short_lookup = lookup_table.head()

In [ ]:
locations_list = []
for i in lookup_table.iterrows():
    position = predicted_keypoints.iloc[i[1]['ImageId']-1][i[1]['FeatureName']]
    locations_list.append(position)
len(locations_list)

In [ ]:
#empty dataframe for testing submission
Submission_df = pd.DataFrame(columns = ['Location'], index = lookup_table.index)

In [ ]:
Submission_df['Location'] = locations_list

In [ ]:
Submission_df.head()

In [ ]:
Submission_df.to_csv('submission_001.csv')